<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/risk/geometric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from pandas_datareader import DataReader as pdr


# get market return data
df = pdr('F-F_Research_Data_Factors','famafrench', start=1900)[1]/100

mkt = df["Mkt-RF"] + df.RF
mkt = pd.DataFrame(mkt).reset_index()
mkt.columns = ['date', 'ret']
mkt.date = mkt.date.astype(str).astype(int)

mkt

,date,ret
0,1927,0.3259
1,1928,0.3895
2,1929,-0.1479
3,1930,-0.2882
4,1931,-0.4404
...,...,...
90,2017,0.2231
91,2018,-0.0512
92,2019,0.3043
93,2020,0.2411


In [ ]:
# get stock return data
tick = 'CVX'        # example stock
close = pdr(tick, "yahoo", start=1970)["Adj Close"]
annual = close.resample('Y').last().pct_change().dropna().reset_index()
annual.columns = ['date', 'ret']
annual.date = annual.date.dt.to_period('Y').astype(str).astype(int)

annual

,date,ret
0,1971,0.106523
1,1972,0.453690
2,1973,-0.090197
3,1974,-0.316027
4,1975,0.417093
5,1976,0.481890
6,1977,-0.051829
7,1978,0.205788
8,1979,0.202667
9,1980,0.764966


In [ ]:
# use market return from 1950 to 2022 as an example

ret = mkt[(mkt.date >= 1950) & (mkt.date <= 2022)].copy()


T = ret.shape[0]            # number of periods
arith = ret.ret.mean()
geom = (1 + ret.ret).prod() ** (1 / T) - 1
print('The arithmetic average of the annual market return from 1950 to 2022 is ', arith, end='.\n')
print('The geometric average of the annual market return from 1950 to 2022 is ', geom, end='.\n')

The arithmetic average of the annual market return from 1950 to 2022 is  0.13057638888888887.
The geometric average of the annual market return from 1950 to 2022 is  0.1163491792071425.
